# **SENet**
此份程式碼會介紹如何使用 tf.keras 的方式建構 SENet 的模型架構。

<img src="https://i.imgur.com/3xGwreb.png" width=1000/>

- [source paper](https://arxiv.org/abs/1709.01507)

## 匯入套件

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Tensorflow 相關套件
import tensorflow as tf
from tensorflow.keras import datasets, layers, Model, Sequential, losses

## 載入資料集

In [ ]:
(x_train, y_train), (x_test, y_test) = datasets.mnist.load_data()

# Expand dimensions
x_train = tf.expand_dims(x_train, axis=3, name=None)
x_test = tf.expand_dims(x_test, axis=3, name=None)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Grayscale to RGB
x_train = tf.repeat(x_train, 3, axis=3)
x_test = tf.repeat(x_test, 3, axis=3)
print(f'x_train shape: {x_train.shape}')
print(f'x_test shape: {x_test.shape}')
print('----------')

# Split dataset into training and validation data
x_val = x_train[int(x_train.shape[0]*0.8):, :, :, :]
y_val = y_train[int(y_train.shape[0]*0.8):]
x_train = x_train[:int(x_train.shape[0]*0.8), :, :, :]
y_train = y_train[:int(y_train.shape[0]*0.8)]
print(f'x_train shape: {x_train.shape}, x_val shape: {x_val.shape}')
print(f'y_train shape: {y_train.shape}, y_val shape: {y_val.shape}')

## SENet Arhietecture

<img src="https://i.imgur.com/mvqWU9g.png" width=1000/>

- [source paper](https://arxiv.org/abs/1709.01507)

In [ ]:
labels_num = 10

In [ ]:
def add_conv(x, filters_num, kernel_size):
    x = layers.Conv2D(filters_num,
                      (kernel_size, kernel_size),
                      strides=1, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Activation('relu')(x)
    x = layers.MaxPooling2D((2, 2), strides=1,
                            padding='same')(x)
    return x

filters_num = 64

inputs = layers.Input(shape=x_train.shape[1:])
x = layers.Resizing(224, 224,
                    interpolation="bilinear",
                    input_shape=x_train.shape[1:])(inputs)
conv_1 = add_conv(x, filters_num, 7)
conv_2 = add_conv(conv_1, filters_num, 3)

squeeze = layers.GlobalAveragePooling2D()(conv_2)
squeeze = layers.Reshape((1, 1, filters_num))(squeeze)
excitation = layers.Dense(filters_num, activation='relu')(squeeze)
excitation = layers.Dense(filters_num, activation='sigmoid')(excitation)

scale = conv_2 * excitation
scale = layers.GlobalAveragePooling2D()(scale)
outputs = layers.Dense(labels_num)(scale)

In [ ]:
SENet_model = Model(inputs=inputs, outputs=outputs)

In [ ]:
SENet_model.summary()

In [ ]:
batch_size = 4
inputs = np.ones((batch_size, x_train.shape[1], x_train.shape[2], 3),
                 dtype=np.float32)
SENet_model(inputs).shape

In [ ]:
SENet_model(inputs)